In [1]:
#https://github.com/wolverton-research-group/qmpy/blob/master/qmpy/data/thermodata/ssub.dat
import pandas as pd
df=pd.read_csv('ssub.dat',sep=' ',header=None)
df.columns=['formula','formula_energy']

In [2]:
df

,formula,formula_energy
0,AlKO6Si2,-3.144727e+00
1,Al2H4O9Si2,-2.500142e+00
2,Al2H4O9Si2,-2.499380e+00
3,AlHO2,-2.589914e+00
4,AlHO2,-2.566077e+00
...,...,...
2085,AsS,-1.763221e-01
2086,As7Re3,-9.887016e-02
2087,As8Ni11,-4.222273e-01
2088,Au,1.605726e-09


In [3]:
#https://stackoverflow.com/questions/12497402/remove-duplicates-by-columns-a-keeping-the-row-with-the-highest-value-in-column
df=df.sort_values('formula_energy', ascending=True).drop_duplicates('formula').sort_index()
df=df.sample(frac=1)
df['id']=df.index

In [4]:
df

,formula,formula_energy,id
1982,AsK3O4,-2.161922,1982
517,CdN2O6,-0.526223,517
1800,Re5Si3,-0.203811,1800
787,CrSi,-0.367932,787
2025,B2V3,-0.629646,2025
...,...,...,...
1349,Li2O3Si,-2.849311,1349
1621,O2Se,-0.779048,1621
1705,O5Rb2Si2,-2.849610,1705
1684,O4Pb2Si,-2.040088,1684


In [5]:
df.to_csv('ssub.csv',index=False)

In [6]:
from jarvis.db.jsonutils import dumpjson
#for jarvis-tools
mem=[]
for i,ii in df.iterrows():
    info={}
    info['id']=ii['id']
    info['formula_energy']=ii['formula_energy']
    info['formula']=ii['formula']
    mem.append(info)
dumpjson(filename='ssub.json',data=mem)

In [7]:
test_size = 0.2
val_size = 0.2
n_train = int(len(df)*(1-(val_size+test_size)))
n_val = int(len(df)*val_size)
n_test = int(len(df)*test_size)
#for leaderboard
mem={}
train={}
val={}
test={}
train_df = df[:n_train]
val_df = df[n_train:n_train+n_val]
test_df = df[n_train+n_val:n_train+n_val+n_test]
for i,ii in train_df.iterrows():
    train[str(ii['id'])]=ii['formula_energy']
for i,ii in val_df.iterrows():
    val[str(ii['id'])]=ii['formula_energy']
for i,ii in test_df.iterrows():
    test[str(ii['id'])]=ii['formula_energy']
mem['train']=train
mem['val']=val
mem['test']=test
fname='ssub_formula_energy.json'
dumpjson(data=mem,filename=fname)

In [8]:
len(train_df), len(val_df), len(test_df)

(1036, 345, 345)

In [9]:
import data_utils
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

elements = ['H', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'K',
 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se',
 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In',
 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd',
 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au',
 'Hg', 'Tl', 'Pb', 'Bi', 'Ac','Th', 'Pa', 'U', 'Np', 'Pu']

def formula_to_ef(df):
    df['comp_dict'] = df['formula'].apply(lambda x: data_utils.parse_formula(x))
    df['comp_fractions'] = df['comp_dict'].apply(lambda x: data_utils.get_fractions(x))
    df = df[~df['comp_fractions'].isnull()]

    for i,e in enumerate(elements):
        df[e] = [ x[i] for x in df['comp_fractions']]

    X_df = np.array(df[elements],dtype='float')    
    y_df = np.array(df['formula_energy'],dtype='float')
    list_df = list(df['id'])

    return X_df, y_df, list_df

In [10]:
X_train, y_train, train_ids = formula_to_ef(train_df)
X_val, y_val, val_ids = formula_to_ef(val_df)
X_test, y_test, test_ids = formula_to_ef(test_df)

/tmp/ipykernel_79452/1972034185.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comp_dict'] = df['formula'].apply(lambda x: data_utils.parse_formula(x))
/tmp/ipykernel_79452/1972034185.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comp_fractions'] = df['comp_dict'].apply(lambda x: data_utils.get_fractions(x))
/tmp/ipykernel_79452/1972034185.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [11]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import add
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

2023-05-14 18:50:04.453480: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 18:50:13.256618: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-14 18:50:13.258398: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-14 18:50:13.258432: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [19]:
#Network from "Enabling deeper learning on big data for materials informatics applications"
#Link to paper: https://dl.acm.org/doi/10.1145/3292500.3330703 https://www.nature.com/articles/s41598-021-83193-1
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = BatchNormalization()(layer_1)
layer_1 = Activation('relu')(layer_1)

fcc_1 = Dense(1024)(in_layer)
gsk_1 = add([fcc_1, layer_1])

layer_2 = Dense(1024)(gsk_1)
layer_2 = BatchNormalization()(layer_2)
layer_2 = Activation('relu')(layer_2)

gsk_2 = add([gsk_1, layer_2])

layer_3 = Dense(1024)(gsk_2)
layer_3 = BatchNormalization()(layer_3)
layer_3 = Activation('relu')(layer_3)

gsk_3 = add([gsk_2, layer_3])

layer_4 = Dense(1024)(gsk_3)
layer_4 = BatchNormalization()(layer_4)
layer_4 = Activation('relu')(layer_4)

gsk_4 = add([gsk_3, layer_4])

layer_5 = Dense(512)(gsk_4)
layer_5 = BatchNormalization()(layer_5)
layer_5 = Activation('relu')(layer_5)

mcc_5 = Dense(512)(gsk_4)
msk_5 = add([mcc_5, layer_5])

layer_6 = Dense(512)(msk_5)
layer_6 = BatchNormalization()(layer_6)
layer_6 = Activation('relu')(layer_6)

msk_6 = add([msk_5, layer_6])

layer_7 = Dense(512)(msk_6)
layer_7 = BatchNormalization()(layer_7)
layer_7 = Activation('relu')(layer_7)

msk_7 = add([msk_6, layer_7])

layer_8 = Dense(256)(msk_7)
layer_8 = BatchNormalization()(layer_8)
layer_8 = Activation('relu')(layer_8)

mcc_8 = Dense(256)(msk_7)
msk_8 = add([mcc_8, layer_8])

layer_9 = Dense(256)(msk_8)
layer_9 = BatchNormalization()(layer_9)
layer_9 = Activation('relu')(layer_9)

msk_9 = add([msk_8, layer_9])

layer_10 = Dense(256)(msk_9)
layer_10 = BatchNormalization()(layer_10)
layer_10 = Activation('relu')(layer_10)

msk_10 = add([msk_9, layer_10])

layer_11 = Dense(128)(layer_10)
layer_11 = BatchNormalization()(layer_11)
layer_11 = Activation('relu')(layer_11)

mcc_11 = Dense(128)(msk_10)
msk_11 = add([mcc_11, layer_11])

layer_12 = Dense(128)(msk_11)
layer_12 = BatchNormalization()(layer_12)
layer_12 = Activation('relu')(layer_12)

msk_12 = add([msk_11, layer_12])

layer_13 = Dense(128)(msk_12)
layer_13 = BatchNormalization()(layer_13)
layer_13 = Activation('relu')(layer_13)

msk_13 = add([msk_12, layer_13])

layer_14 = Dense(64)(msk_13)
layer_14 = BatchNormalization()(layer_14)
layer_14 = Activation('relu')(layer_14)

mcc_14 = Dense(64)(msk_13)
msk_14 = add([mcc_14, layer_14])

layer_15 = Dense(64)(msk_14)
layer_15 = BatchNormalization()(layer_15)
layer_15 = Activation('relu')(layer_15)

msk_15 = add([msk_14, layer_15])

layer_16 = Dense(32)(msk_15)
layer_16 = BatchNormalization()(layer_16)
layer_16 = Activation('relu')(layer_16)

mcc_16 = Dense(32)(msk_15)
msk_16 = add([mcc_16, layer_16])

out_layer = Dense(1)(msk_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


In [20]:
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
33/33 - 12s - loss: 7.7376 - mean_absolute_error: 7.7376 - val_loss: 1.1927 - val_mean_absolute_error: 1.1927 - 12s/epoch - 355ms/step
Epoch 2/3000
33/33 - 1s - loss: 1.1213 - mean_absolute_error: 1.1213 - val_loss: 0.9184 - val_mean_absolute_error: 0.9184 - 923ms/epoch - 28ms/step
Epoch 3/3000
33/33 - 1s - loss: 0.9265 - mean_absolute_error: 0.9265 - val_loss: 0.8819 - val_mean_absolute_error: 0.8819 - 948ms/epoch - 29ms/step
Epoch 4/3000
33/33 - 1s - loss: 0.7525 - mean_absolute_error: 0.7525 - val_loss: 0.8916 - val_mean_absolute_error: 0.8916 - 849ms/epoch - 26ms/step
Epoch 5/3000
33/33 - 1s - loss: 0.8392 - mean_absolute_error: 0.8392 - val_loss: 0.8191 - val_mean_absolute_error: 0.8191 - 896ms/epoch - 27ms/step
Epoch 6/3000
33/33 - 1s - loss: 0.8107 - mean_absolute_error: 0.8107 - val_loss: 0.9774 - val_mean_absolute_error: 0.9774 - 888ms/epoch - 27ms/step
Epoch 7/3000
33/33 - 1s - loss: 0.6539 - mean_absolute_error: 0.6539 - val_loss: 0.9781 - val_mean_absolute_erro

In [21]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

11/11 [==============================] - 0s 5ms/step
0.14187669189620658


In [22]:
f=open('SinglePropertyPrediction-test-formula_energy-ssub-AI-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k)+'\n'
    f.write(line)
f.close()